# Project 9 -- Ritvik Indupuri

**TA Help:** John Smith, Alice Jones

- Help with figuring out how to write a function.
    
**Collaboration:** Friend1, Friend2
    
- Helped figuring out how to load the dataset.
- Helped debug error with my plot.

## Question 1

In [1]:
import sqlite3

# Load the read-only Lahman database
read_only_lahman = sqlite3.connect('/anvil/projects/tdm/data/lahman/lahman.db')

# Create a local, writable database file
local_lahman = sqlite3.connect('./lahman.db')

# Copy the data from the source to our local file
read_only_lahman.backup(local_lahman)

# Close the connections
read_only_lahman.close()
local_lahman.close()

In [2]:
%sql sqlite:///./lahman.db

In [3]:
%%sql
CREATE TRIGGER IF NOT EXISTS update_yearID
AFTER INSERT ON Teams
FOR EACH ROW
BEGIN
    UPDATE Teams
    SET yearID = strftime('%Y', 'now')
    WHERE teamID = NEW.teamID;
END;

 * sqlite:///./lahman.db
Done.


[]

In [4]:
%%sql
INSERT INTO Teams (teamID, name, yearID)
VALUES ('team1', 'Test Team', 0);

 * sqlite:///./lahman.db
1 rows affected.


[]

In [5]:
%%sql
SELECT * FROM Teams WHERE teamID = 'team1';

 * sqlite:///./lahman.db
Done.


yearID,lgID,teamID,franchID,divID,teamRank,G,Ghome,W,L,DivWin,WCWin,LgWin,WSWin,R,AB,H,2B,3B,HR,BB,SO,SB,CS,HBP,SF,RA,ER,ERA,CG,SHO,SV,IPouts,HA,HRA,BBA,SOA,E,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
2025,None,team1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Test Team,None,None,None,None,None,None,None


This question introduced Triggers—automatic actions triggered by events like INSERT, UPDATE, or DELETE. I created a trigger called update_yearID that fires after any INSERT on the Teams table. It uses NEW.teamID to reference the inserted row. My test in 1.3 confirmed it worked: even though I inserted a yearID of 0, the trigger updated it to 2025.


## Question 2

In [6]:
%%sql
DROP TABLE IF EXISTS DeathRecords;
CREATE TABLE DeathRecords (
    playerID TEXT NOT NULL,
    yearID INTEGER NOT NULL,
    monthID INTEGER NOT NULL,
    dayID INTEGER NOT NULL
);

 * sqlite:///./lahman.db
Done.
Done.


[]

In [7]:
%%sql
CREATE TRIGGER IF NOT EXISTS update_people_death
AFTER INSERT ON DeathRecords
FOR EACH ROW
BEGIN
    UPDATE People
    SET deathYear = NEW.yearID,
        deathMonth = NEW.monthID,
        deathDay = NEW.dayID
    WHERE playerID = NEW.playerID;
END;

 * sqlite:///./lahman.db
Done.


[]

In [8]:
%%sql
INSERT INTO DeathRecords (playerID, yearID, monthID, dayID)
VALUES ('abreubo01', 2025, 1, 1);

 * sqlite:///./lahman.db
1 rows affected.


[]

In [9]:
%%sql
SELECT playerID, deathYear, deathMonth, deathDay 
FROM People 
WHERE playerID = 'abreubo01';

 * sqlite:///./lahman.db
Done.


playerID,deathYear,deathMonth,deathDay
abreubo01,2025,1,1


In [10]:
%%sql
INSERT INTO DeathRecords (playerID, yearID, monthID, dayID)
VALUES
    ('pintore01', 2025, 2, 1),
    ('hundlni01', 2025, 3, 1),
    ('detwiro01', 2025, 4, 1);

 * sqlite:///./lahman.db
3 rows affected.


[]

In [11]:
%%sql
SELECT playerID, deathYear, deathMonth, deathDay 
FROM People 
WHERE playerID IN ('pintore01', 'hundlni01', 'detwiro01');

 * sqlite:///./lahman.db
Done.


playerID,deathYear,deathMonth,deathDay
detwiro01,2025,4,1
hundlni01,2025,3,1
pintore01,2025,2,1


This question showed a practical use of triggers. I created a DeathRecords table to log player deaths, then added a trigger that updates the People table whenever a new death is recorded. This keeps data synced automatically—adding to one table updates the other. My verification queries confirmed it worked for both single and multiple inserts.


## Question 3

In [12]:
%%sql
CREATE VIEW IF NOT EXISTS TallPlayers AS
SELECT playerID, nameFirst, nameLast, height, weight
FROM People
WHERE height > 72;

 * sqlite:///./lahman.db
Done.


[]

In [13]:
%%sql
SELECT * FROM TallPlayers LIMIT 5;

 * sqlite:///./lahman.db
Done.


playerID,nameFirst,nameLast,height,weight
aardsda01,David,Aardsma,75,215
aaronto01,Tommie,Aaron,75,190
aasedo01,Don,Aase,75,190
abadan01,Andy,Abad,73,184
abadfe01,Fernando,Abad,74,235


In [14]:
%%sql
CREATE VIEW IF NOT EXISTS TallHeavyPlayers AS
SELECT playerID, nameFirst, nameLast, height, weight
FROM People
WHERE height > 72
  AND weight IS NOT NULL
  AND birthYear = 1994
ORDER BY weight DESC;

 * sqlite:///./lahman.db
Done.


[]

In [15]:
%%sql
SELECT * FROM TallHeavyPlayers LIMIT 5;

 * sqlite:///./lahman.db
Done.


playerID,nameFirst,nameLast,height,weight
castidi01,Diego,Castillo,75,268
perdoan01,Angel,Perdomo,80,265
ponceco01,Cody,Ponce,78,255
quezajo01,Johan,Quezada,81,255
wellsty01,Tyler,Wells,80,255


In [16]:
%%sql
SELECT * FROM TallHeavyPlayers LIMIT 7;

 * sqlite:///./lahman.db
Done.


playerID,nameFirst,nameLast,height,weight
castidi01,Diego,Castillo,75,268
perdoan01,Angel,Perdomo,80,265
ponceco01,Cody,Ponce,78,255
quezajo01,Johan,Quezada,81,255
wellsty01,Tyler,Wells,80,255
alonspe01,Pete,Alonso,74,253
bednada01,David,Bednar,73,250


This question introduced Views which essentially is stored queries that act like tables. My TallPlayers view filters players over 72 inches, while TallHeavyPlayers handles a more complex filter. Instead of retyping long queries, I just SELECT from the view. As shown in 3.4 and 3.5, I can easily apply LIMITs, making my queries cleaner and reusable.

## Question 4

In [17]:
%config SqlMagic.autocommit = False

In [18]:
%%sql
SAVEPOINT insert_deathrecords;
INSERT OR IGNORE INTO DeathRecords (playerID, yearID, monthID, dayID) VALUES ('bandyje01', 2025, 5, 1);
INSERT OR IGNORE INTO DeathRecords (playerID, yearID, monthID, dayID) VALUES ('meyeral01', 2025, 6, 1);
INSERT OR IGNORE INTO DeathRecords (playerID, yearID, monthID, dayID) VALUES ('perdoan01', 2025, 7, 1);
RELEASE insert_deathrecords;
COMMIT;

 * sqlite:///./lahman.db
Done.
1 rows affected.
1 rows affected.
1 rows affected.
Done.
(sqlite3.OperationalError) cannot commit - no transaction is active
[SQL: COMMIT;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [19]:
%%sql
SELECT * FROM DeathRecords WHERE playerID IN ('bandyje01', 'meyeral01', 'perdoan01');

 * sqlite:///./lahman.db
Done.


playerID,yearID,monthID,dayID
bandyje01,2025,5,1
meyeral01,2025,6,1
perdoan01,2025,7,1


In [20]:
%%sql
BEGIN;
SAVEPOINT deathrecords_insert2;
-- This first insert is successful (I'm using 'castidi02' as suggested)
INSERT OR IGNORE INTO DeathRecords (playerID, yearID, monthID, dayID) VALUES ('castidi02', 2025, 8, 1);
-- This second insert will fail silently because dayID is NULL
INSERT OR IGNORE INTO DeathRecords (playerID, yearID, monthID, dayID) VALUES ('quezajo01', 2025, 9, NULL);
-- Here, I roll back all changes made since the savepoint
ROLLBACK TO deathrecords_insert2;
COMMIT;

 * sqlite:///./lahman.db
Done.
Done.
1 rows affected.
0 rows affected.
Done.
Done.


[]

In [21]:
%%sql
SELECT * FROM DeathRecords WHERE playerID IN ('castidi02', 'quezajo01');

 * sqlite:///./lahman.db
Done.


playerID,yearID,monthID,dayID


This question focused on Transactions, which protect data integrity by using an "all or nothing" method. I began by disabling autocommit with %config SqlMagic.autocommit = False.
In transaction 4.1, all three inserts were valid, so I used RELEASE (similar to COMMIT) to save them. The query in 4.2 confirmed success—all rows were stored.

In transaction 4.3, I inserted one valid row ('castidi02') and one invalid row ('quezajo01'). Since the second insert failed, I used ROLLBACK TO to undo all changes, including the valid one. The query in 4.4 returned an empty set, showing that neither row was saved. This demonstrates how transactions prevent partial or faulty data from being committed.


## Question 5

In [22]:
%%sql
CREATE TRIGGER IF NOT EXISTS update_myplayers_death
AFTER INSERT ON DeathRecords
FOR EACH ROW
BEGIN
    UPDATE People
    SET deathYear = NEW.yearID,
        deathMonth = NEW.monthID,
        deathDay = NEW.dayID
    WHERE playerID = NEW.playerID;
END;

 * sqlite:///./lahman.db
Done.


[]

In [23]:
%%sql
CREATE VIEW IF NOT EXISTS DeceasedPlayers AS
SELECT playerID, nameFirst, nameLast, deathYear, deathMonth, deathDay
FROM People
WHERE deathYear IS NOT NULL;

 * sqlite:///./lahman.db
Done.


[]

In [24]:
%%sql
SAVEPOINT final_insert;
INSERT OR IGNORE INTO DeathRecords (playerID, yearID, monthID, dayID) VALUES ('johndoe01', 2025, 10, 1);
INSERT OR IGNORE INTO DeathRecords (playerID, yearID, monthID, dayID) VALUES ('janedoe01', 2025, 11, 1);
INSERT OR IGNORE INTO DeathRecords (playerID, yearID, monthID, dayID) VALUES ('foobar01', 2025, 12, 1);
RELEASE final_insert;
COMMIT;

 * sqlite:///./lahman.db
Done.
1 rows affected.
1 rows affected.
1 rows affected.
Done.
(sqlite3.OperationalError) cannot commit - no transaction is active
[SQL: COMMIT;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [25]:
%%sql
SELECT * FROM DeathRecords WHERE playerID IN ('johndoe01', 'janedoe01', 'foobar01');

 * sqlite:///./lahman.db
Done.


playerID,yearID,monthID,dayID
johndoe01,2025,10,1
janedoe01,2025,11,1
foobar01,2025,12,1


In [26]:
%%sql
SELECT * FROM DeceasedPlayers WHERE playerID IN ('johndoe01', 'janedoe01', 'foobar01');

 * sqlite:///./lahman.db
Done.


playerID,nameFirst,nameLast,deathYear,deathMonth,deathDay


This final question tied together all three concepts.
In Transaction 5.3, I safely inserted 3 new players into the DeathRecords table. Query 5.4 confirmed the inserts worked.
Right after committing, the Trigger from 5.1 fired 3 times, updating the People table with death info for each player.
Since the People table changed, my View (DeceasedPlayers from 5.2) updated automatically. Query 5.5 then pulled all 3 new players from the view.


## Pledge

By submitting this work I hereby pledge that this is my own, personal work. I've acknowledged in the designated place at the top of this file all sources that I used to complete said work, including but not limited to: online resources, books, and electronic communications. I've noted all collaboration with fellow students and/or TA's. I did not copy or plagiarize another's work.

> As a Boilermaker pursuing academic excellence, I pledge to be honest and true in all that I do. Accountable together – We are Purdue.